In [1]:
from gensim import corpora, models
import spacy

In [2]:
nlp = spacy.load('en_core_web_sm')

**Vector transformations in Gensim**

Now that we know what vector transformations are, let's get used to creating them, and using them. We will be performing these transformations with Gensim, but even scikit-learn can be used. We'll also have a look at scikit-learn's approach later on.

Let's create our corpus now. We discussed earlier that a corpus is a collection of documents. In our examples, each document would just be one sentence, but this is obviously not the case in most real-world examples we will be dealing with. We should also note that once we are done with preprocessing, we get rid of all punctuation marks - as for as our vector representation is concerned, each document is just one sentence.

Of course, before we start, be sure to install Gensim. Like spaCy, pip or conda is the best way to do this based on your working environment.

In [3]:
# We performed very similar preprocessing when we introduced spaCy. What do our documents look like now?
documents = [u"Football club Arsenal defeat local rivals this weekend.", 
             u"Weekend football frenzy takes over London.", 
             u"Bank open for take over bids after losing millions.", 
             u"London football clubs bid to move to Wembley stadium.", 
             u"Arsenal bid 50 million pounds for striker Kane.", 
             u"Financial troubles result in loss of millions for bank.", 
             u"Western bank files for bankruptcy after financial losses.", 
             u"London football club is taken over by oil millionaire from Russia.", 
             u"Banking on finances not working for Russia."]

texts = []
for document in documents:
    doc = nlp(document)
    text = [
        w.lemma_
        for w in doc
        if not w.is_stop and not w.is_punct and not w.like_num
    ]
    texts.append(text)

dictionary = corpora.Dictionary(texts)

Let's start by whipping up a bag-of-words representation for our mini-corpus. Gensim allows us to do this very conveniently through its `dictionary` class.

In [4]:
print(dictionary.token2id)

{'Arsenal': 0, 'club': 1, 'defeat': 2, 'football': 3, 'local': 4, 'rival': 5, 'weekend': 6, 'London': 7, 'frenzy': 8, 'take': 9, 'bank': 10, 'bid': 11, 'lose': 12, 'million': 13, 'open': 14, 'Wembley': 15, 'stadium': 16, 'Kane': 17, 'arsenal': 18, 'pound': 19, 'striker': 20, 'financial': 21, 'loss': 22, 'result': 23, 'trouble': 24, 'bankruptcy': 25, 'file': 26, 'western': 27, 'Russia': 28, 'millionaire': 29, 'oil': 30, 'banking': 31, 'finance': 32, 'work': 33}


There are 32 unique words in our corpus, all of which are represented in our dictionary with each word being assigned an index value. When we refer to a word's word_id henceforth, it means we are talking about the words integer-id mapping made by the dictionary.

We will be using the `doc2bow` method, which, as the name suggests, helps convert our document to bag-of-words.

In [5]:
corpus = [dictionary.doc2bow(text) for text in texts] 

If we print our corpus, we'll have our bag of words representation of the documents we used.

In [6]:
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)],
 [(3, 1), (6, 1), (7, 1), (8, 1), (9, 1)],
 [(10, 1), (11, 1), (12, 1), (13, 1), (14, 1)],
 [(1, 1), (3, 1), (7, 1), (11, 1), (15, 1), (16, 1)],
 [(11, 1), (17, 1), (18, 1), (19, 1), (20, 1)],
 [(10, 1), (13, 1), (21, 1), (22, 1), (23, 1), (24, 1)],
 [(10, 1), (21, 1), (22, 1), (25, 1), (26, 1), (27, 1)],
 [(1, 1), (3, 1), (7, 1), (9, 1), (28, 1), (29, 1), (30, 1)],
 [(28, 1), (31, 1), (32, 1), (33, 1)]]

This is a list of lists, where each individual list represents a documents bag-of-words representation. A reminder: you might see different numbers in your list, this is because each time you create a dictionary, different mappings will occur. Unlike the example we demonstrated, where an absence of a word was a 0, we use tuples that represent (word_id, word_count). We can easily verify this by checking the original sentence, mapping each word to its integer ID and reconstructing our list. We can also notice in this case each document has not greater than one count of each word - in smaller corpuses, this tends to happen.

And voila! Our corpus is assembled, and we are ready to work machine learning/information retrieval magic on them whenever we would like. But before we sink our teeth into it... let's spend some more time with some details regarding corpuses.

We previously mentioned how Gensim is powerful because it uses streaming corpuses. But in this case, the entire list is loaded into the RAM. This is not a bother for us because it is a toy example, but in any real-world cases, this might cause problems. How do we get past this?

We can start by storing the corpus, once it is created, to disk. One way to do this is as follows:

```python
    corpora.MmCorpus.serialize('/tmp/example.mm', corpus)
```

By storing the corpus to disk and then later loading from disk, we are being far more memory efficient, because at most one vector resides in the RAM at a time. The Gensim tutorial [13] on corpora and vector spaces covers a little more than what we discussed so far and may be useful for some readers.

Converting a bag of words representation into TF-IDF, for example, is also made very easy with Gensim. We first choose the model/representation we want from the Gensim models directory.

In [7]:
# TF-IDF representation

tfidf = models.TfidfModel(corpus)

This means that `tfidf` now represents a TF-IDF table **trained** on our corpus. Note that in case of TFIDF, the training consists simply of going through the supplied corpus once and computing document frequencies of all its features. Training other models, such as latent semantic analysis or latent dirichlet allocation, is much more involved and, consequently, takes much more time. We will explore those transformations on the chapters on topic modelling. It is also important to note that all such vector transformations require the same input feature space - which means the same dictionary (and of course, vocabulary).

So, what does a TF-IDF representation of our corpus look like? All we have to do is this:

In [9]:
for document in tfidf[corpus]:
    print(document)

[(0, 0.4538520228951382), (1, 0.2269260114475691), (2, 0.4538520228951382), (3, 0.1675032779320012), (4, 0.4538520228951382), (5, 0.4538520228951382), (6, 0.3106776504135697)]
[(3, 0.2421296766697527), (6, 0.44909138478886224), (7, 0.32802654645398593), (8, 0.6560530929079719), (9, 0.44909138478886224)]
[(10, 0.29019840161676663), (11, 0.29019840161676663), (12, 0.5803968032335333), (13, 0.3973019972146358), (14, 0.5803968032335333)]
[(1, 0.29431054749542984), (3, 0.21724253258131515), (7, 0.29431054749542984), (11, 0.29431054749542984), (15, 0.5886210949908597), (16, 0.5886210949908597)]
[(11, 0.24253562503633302), (17, 0.48507125007266605), (18, 0.48507125007266605), (19, 0.48507125007266605), (20, 0.48507125007266605)]
[(10, 0.2615055248879334), (13, 0.35801943340074827), (21, 0.35801943340074827), (22, 0.35801943340074827), (23, 0.5230110497758668), (24, 0.5230110497758668)]
[(10, 0.24434832234965204), (21, 0.33453001789363906), (22, 0.33453001789363906), (25, 0.4886966446993041), 

If you remember what we said about TF-IDF, you will be able to identify the float next to each word_id - it is the product of the TF and IDF scores for that particular word, instead of just the word count which was present before. The higher the score, the more important the word in the document.

We can use this representation as input for our ML algorithms as well, and we can also further chain or link these vector representations by performing another transformation on them.

**n-grams and some more preprocessing**

When working with textual data, context can be very important. As we discussed before, we sometimes lose this context in vector representations, knowing only the count of each word. **N-grams**, and in particular, **bi-grams** are going to help us solve this problem, at least to some extent.

An n-gram is a contiguous sequence of n items in the text. In our case, we will be dealing with words being the item, but depending on the use case, it could be even letters, syllables, or sometimes in the case of speech, phonemes. A bi-gram is when $n = 2$.

One way bi-grams are calculated in the text is by calculating the conditional probability of a token given by the preceding token. It can also just be calculated by choosing words that appear next to each other, but it is more useful for us to use bi-grams that are more likely to appear as a pair. Such a bi-gram is called a collocation. What this means is that we're trying to find pairs of words that are more likely to appear around each other. For example, New York or Machine Learning could be two possible pairs of words created by bi-grams. In other words, based on the training data (usually the corpus), we identify that it is with high probability that the word York follows the word New, and that it is worth considering New York as one identity. We must be careful to get rid of stop words before running a bi-gram model on our corpus, as there could be meaningless bi-grams formed. The Gensim bi-gram model is basically an implementation of collocation identification. 

We can clearly see how this is useful - we can now pick up phrases from our corpus, and New York certainly provides us with more information than the words New and York separately. This means it can be added to our preprocessing pipeline. 

Gensim approaches bigrams by simply combining the two high probability tokens with an underscore. The tokens new and york will now become new_york instead. Similar to the TF-DF model, bigrams can be created using another Gensim model - `Phrases`.

In [10]:
# creating n-grams
bigram = models.Phrases(texts) 
texts = [bigram[line] for line in texts]

In [11]:
texts

[['football', 'club', 'Arsenal', 'defeat', 'local', 'rival', 'weekend'],
 ['weekend', 'football', 'frenzy', 'take', 'London'],
 ['bank', 'open', 'bid', 'lose', 'million'],
 ['London', 'football', 'club', 'bid', 'Wembley', 'stadium'],
 ['arsenal', 'bid', 'pound', 'striker', 'Kane'],
 ['financial', 'trouble', 'result', 'loss', 'million', 'bank'],
 ['western', 'bank', 'file', 'bankruptcy', 'financial', 'loss'],
 ['London', 'football', 'club', 'take', 'oil', 'millionaire', 'Russia'],
 ['banking', 'finance', 'work', 'Russia']]

Each line will now have all possible bi-grams created. It should be noted that in our toy example, we will have no bi-grams or meaningless bi-grams being created. To see an example where bi-grams provide useful information, the Jupyter notebook [14] written by me on topic modeling serves well.

Since by creating new phrases we add words to our dictionary, this step must be done before we create our dictionary. We would have to run this:

In [33]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [34]:
print(dictionary.token2id)

{'Arsenal': 0, 'club': 1, 'defeat': 2, 'football': 3, 'local': 4, 'rival': 5, 'weekend': 6, 'London': 7, 'frenzy': 8, 'take': 9, 'bank': 10, 'bid': 11, 'lose': 12, 'million': 13, 'open': 14, 'Wembley': 15, 'stadium': 16, 'Kane': 17, 'arsenal': 18, 'pound': 19, 'striker': 20, 'financial': 21, 'loss': 22, 'result': 23, 'trouble': 24, 'bankruptcy': 25, 'file': 26, 'western': 27, 'Russia': 28, 'millionaire': 29, 'oil': 30, 'banking': 31, 'finance': 32, 'work': 33}


After we are done creating our bi-grams, we can create tri-grams, and other n-grams by simply running the phrases model multiple times on our corpus. Bi-grams still remains the most used n-gram model, though it is worth one's time to glance over the other uses and kinds of n-gram implementations. Again, the Wikipedia page [15] serves as a good introductory resource.

This brings us to the end of the preprocessing techniques covered in this book. It must be noted however that there is no one perfect preprocessing pipeline or set of rules - it depends largely on our use-cases, the kind of data we are working with, and what sort of information we wish to preserve (or lose!).

For example, one popular preprocessing technique involves removing both high frequency and low-frequency words. We can do this in Gensim with the `dictionary` module. Let's say we would like to get rid of words that occur in less than 20 documents, or in more than 50% of the documents, we would add the following:

In [35]:
dictionary.filter_extremes(no_below=20, no_above=0.5) 

In [36]:
print(dictionary)

Dictionary<0 unique tokens: []>


We can also remove most frequent tokens or prune out certain token ids. You can refer to the docs [16] to see the full extent of the preprocessing tools the dictionary class can provide us.

More often than not, it's after multiple iterations of preprocessing and running our algorithms when we figure out the correct preprocessing techniques we wish to use. What is important for us is to know what kind of tools are available to do this, and what is the reason behind doing all of this.